In [3]:
!wget -O mistral-7b-instruct-v0.1.Q4_K_M.gguf https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf


--2025-06-20 07:44:47--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1750407848&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDQwNzg0OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2Yy

In [5]:
# 📌 Установка ctransformers
!pip install -q ctransformers

# 📌 Импорты
import numpy as np
import json
import torch
from ctransformers import AutoModelForCausalLM

# 📌 Загрузка эмбеддингов и текстов
with open("chunk_ids.json", encoding="utf-8") as f:
    ids = json.load(f)

with open("cleaned_text_chunks.json", encoding="utf-8") as f:
    chunks = json.load(f)

embeddings = np.load("chunk_embeddings.npy")
id2text = {chunk["id"]: chunk["text"] for chunk in chunks}

# 📌 Функция поиска по эмбеддингу
from sentence_transformers import SentenceTransformer, util

model_embed = SentenceTransformer("intfloat/multilingual-e5-base")

def search(query, top_k=5):
    query_embedding = model_embed.encode(f"query: {query}", convert_to_tensor=True, normalize_embeddings=True)

    # 👇 Конвертация np.ndarray -> torch.Tensor и перенос на тот же девайс
    corpus_tensor = torch.tensor(embeddings, dtype=torch.float32, device=query_embedding.device)

    scores = util.cos_sim(query_embedding, corpus_tensor)[0]
    top_idx = scores.argsort(descending=True)[:top_k]
    return [{"id": ids[i], "score": float(scores[i]), "text": id2text[ids[i]]} for i in top_idx]


# 📌 Загрузка локальной LLM (Mistral)
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=0  # 0 — запуск на CPU
)

# 📌 Генерация ответа
def generate_answer(query, context_chunks):
    # Обрезаем текст каждого чанка до 200 символов
    context = "\n".join([f"- {chunk['text'][:200]}" for chunk in context_chunks])
    prompt = f"""Ты — помощник, отвечающий на вопросы на основе предоставленного контекста.
Если ответа нет в контексте, скажи, что не знаешь.

Контекст:
{context}

Вопрос: {query}
Ответ:"""
    response = llm(prompt, max_new_tokens=256)
    return response




Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Пример запроса
query = "Когда появилась первая ЭВМ?"
top_chunks = search(query, top_k=3)
answer = generate_answer(query, top_chunks)

# Вывод
print("🔎 Вопрос:", query)
print("\n📄 Контекст:")
for i, chunk in enumerate(top_chunks, 1):
    print(f"{i}. {chunk['text'][:200]}...\n")  # первые 200 символов для краткости

print("🤖 Ответ:")
print(answer)


🔎 Вопрос: Когда появилась первая ЭВМ?

📄 Контекст:
1. В настоящее время термин ЭВМ, как относящийся больше к вопросам конкретной физической реализации компьютера, почти вытеснен из бытового употребления и в основном используется инженерами цифровой элект...

2. Впервые трактовка слова компьютер появилась в 1897 году в Оксфордском словаре английского языка. Его составители тогда понимали компьютер как механическое вычислительное устройство. В 1946 году словар...

3. 1942 год — в Университете штата Айова Джон Атанасов и его аспирант Клиффорд Берри (англ. Clifford Berry) создали (а точнее — разработали и начали монтировать) первый в США электронный цифровой компьют...

🤖 Ответ:
 1942 


In [7]:
# Пример запроса
query = "Кто такой Норберт Винер?"
top_chunks = search(query, top_k=3)
answer = generate_answer(query, top_chunks)

# Вывод
print("🔎 Вопрос:", query)
print("\n📄 Контекст:")
for i, chunk in enumerate(top_chunks, 1):
    print(f"{i}. {chunk['text'][:200]}...\n")  # первые 200 символов для краткости

print("🤖 Ответ:")
print(answer)


🔎 Вопрос: Кто такой Норберт Винер?

📄 Контекст:
1. Cybernetics: Or Control and Communication in the Animal and the Machine (Кибернетика: Или Контроль и Коммуникация у Животных и Машин) — это книга, которую Норберт Винер опубликовал в 1948. В книге впе...

2. Винер подробно рассказывает, что источником идей его книги является серия встреч в течение десяти лет в Гарвардской медицинской школе, где учёные-медики обсуждали научные методы вместе с математиками,...

3. Винер популяризировал социальные значения кибернетики, проведя аналогии между автоматическими системами (такими как регулируемый паровой двигатель) и человеческими институтами в его бестселлере «Кибер...

🤖 Ответ:
 Норберт Винер — американский математик, физик и кибернетик, крупный специалист в области контроля и коммуникации.


In [8]:
# Пример запроса
query = "Какие бывают сигналы в технике?"
top_chunks = search(query, top_k=3)
answer = generate_answer(query, top_chunks)

# Вывод
print("🔎 Вопрос:", query)
print("\n📄 Контекст:")
for i, chunk in enumerate(top_chunks, 1):
    print(f"{i}. {chunk['text'][:200]}...\n")  # первые 200 символов для краткости

print("🤖 Ответ:")
print(answer)

🔎 Вопрос: Какие бывают сигналы в технике?

📄 Контекст:
1. Железнодорожная сигнализация — это система управления железнодорожным движением в целях избежания столкновений. Из-за использования рельс, которые имеют малое сцепление, железнодорожные поезда чрезвыч...

2. На сигнальной точке установлены датчик пути, вырабатывающий информацию о состоянии лежащего за ним пути, шифратор, в соответствии с этой информацией выбирающий частоту, и путевой генератор, вырабатыва...

3. Наиболее распространены светофоры с сигналами (обычно круглыми) трёх цветов: красного, жёлтого (горит 0,5-1 сек.) и зелёного. В некоторых странах, в том числе и в России, вместо жёлтого используется о...

🤖 Ответ:
 Сигналы — это электрические или световые сигналы, передающие информацию между элементами устройства. Сигналам соответствуют специальные изменения в электричестве или оптике. Например, определяемая частота звука может быть сигналом для управления прибором. Кроме того, как упомянуто выше, во многих случаях испол